In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
from transformers import ViTForImageClassification,ViTConfig
from dataclasses import dataclass
import math

In [3]:
model= ViTForImageClassification.from_pretrained("google/vit-base-patch16-224") #finetuned on imagenet

In [126]:
model_hf_1=model.state_dict()

In [69]:
conf=ViTConfig()
print(conf)

ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.41.0"
}



In [12]:
@dataclass
class ViT_Config():
    image_size = 224
    patch_size = 16
    embed_dim=768
    ff_dim=768*4 
    num_heads=12
    layers=12

conf=ViT_Config()

In [120]:
x=0
for k,v in model_hf.items():
    x=x+1
print(x)

200


In [6]:
import torch
import torch.nn as nn

#torch gelu is different
class NewGELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

In [13]:
class MLP(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
    def forward(self,x):
        x=self.c_fc(x)
        x=self.gelu(x)
        x=self.c_proj(x)
        # self.out=self.dropout(x)
        return x

In [14]:
class NewGELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

#minor changes from the gpt version
class TransformerBlock(nn.Module):
    def __init__(self, max_length, embed_dim, ff_dim, num_heads, dropout=0.1):

        super(TransformerBlock, self).__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisble by num_heads"

        self.max_length = max_length
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.num_heads = num_heads
        self.dp = dropout

        #derv:
        self.head_size = self.embed_dim // self.num_heads

        #attention blocks
        self.query = nn.Linear(self.embed_dim, self.embed_dim)
        self.key = nn.Linear(self.embed_dim, self.embed_dim)
        self.value = nn.Linear(self.embed_dim, self.embed_dim)
        self.c_proj = nn.Linear(self.embed_dim, self.embed_dim)

        #feedforward blocks
        self.mlpf = MLP(conf)
        
        #after attn and ff blocks
        self.dropout = nn.Dropout(self.dp, inplace=True)

        #depends
        self.ln1 = nn.LayerNorm(self.embed_dim)
        self.ln2 = nn.LayerNorm(self.embed_dim)

    def attn(self, x):

        batch_size, seq_length = x.shape[:2]

        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        # import code;code.interact(local=locals())

        Q = Q.view(batch_size, -1, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.num_heads, self.head_size).permute(0, 2, 1, 3)
        # import code;code.interact(local=locals())

        att = torch.einsum('bhqd,bhkd->bhqk', [Q, K])/(self.head_size ** 0.5) #scaled dot produt attention
        att = F.softmax(att, dim=-1)

        out = torch.einsum('bhal,bhlv->bhav', [att, V]).permute(0,2,1,3).contiguous()
        out = out.view(batch_size, -1, self.num_heads * self.head_size)
        out = self.dropout(self.c_proj(out)) #projection after attending to tokens
        return out
    
